In [9]:

import argparse
import os
import time
from loguru import logger

import cv2

import torch

from yolox.data.data_augment import ValTransform
from yolox.data.datasets import COCO_CLASSES
from yolox.exp import get_exp
from yolox.utils import fuse_model , get_model_info , postprocess , vis 

from yolox.models import YOLOX , YOLOPAFPN , YOLOXHead

# モデルの定義

In [15]:
params = {
    "depth":1.33,
    "width":1.25,
    "in_channels":[256,512,1024],
    "act":"silu",
    "num_classes":80
}

backbone = YOLOPAFPN( params["depth"], params["width"],in_channels = params["in_channels"],act = params["act"])
head = YOLOXHead( params["num_classes"],params["width"],in_channels = params["in_channels"],act = params["act"])
model = YOLOX( backbone , head )

In [16]:
pth =torch.load("./pth/yolox_x.pth" , map_location=torch.device('cpu') )
model.load_state_dict( pth["model"] )

<All keys matched successfully>

YOLOX(
  (backbone): YOLOPAFPN(
    (backbone): CSPDarknet(
      (stem): Focus(
        (conv): BaseConv(
          (conv): Conv2d(12, 80, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(80, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
      )
      (dark2): Sequential(
        (0): BaseConv(
          (conv): Conv2d(80, 160, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (1): CSPLayer(
          (conv1): BaseConv(
            (conv): Conv2d(160, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (bn): BatchNorm2d(80, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (act): SiLU(inplace=True)
          )
          (conv2): BaseConv(
            (conv): Conv2d(160, 80, kernel_si